# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

мобильный оператор хочет изучить данные тарифов клиентов.цель проекта:узнать, для кого новые тарифы будут выгодней

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

## Разбейте данные на выборки

In [7]:
features = data.drop('is_ultra',axis=1)
target = data.is_ultra

In [8]:
train, validate, test = \
              np.split(data.sample(frac=1, random_state=42), 
                       [int(.6*len(data)), int(.8*len(data))])

In [9]:
train_features = train.drop('is_ultra',axis=1)
train_target = train.is_ultra
validate_features = validate.drop('is_ultra',axis=1)
validate_target = validate.is_ultra
test_features = test.drop('is_ultra',axis=1)
test_target = test.is_ultra

## Исследуйте модели  

обучим модель на тренировочной выборке 

проверим на модели решающего дерева

### модель решающего дерева 

In [10]:
best_result = 0
best_leaf = 0
for depth in range(1,16):
    for leaf in range(1,11):
        model_decision_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_leaf=leaf)
        model_decision_tree.fit(train_features, train_target)
        predictions_valid = model_decision_tree.predict(validate_features)
        result = accuracy_score(validate_target, predictions_valid)
        if result > best_result:
            best_result = result
            best_leaf = leaf
print('min_samples_leaf =', leaf)
print('max_depth =', depth, ':',best_result)


min_samples_leaf = 10
max_depth = 15 : 0.8087091757387247


доля правильных ответов самая высокая при глубине дерева 15 и минимальном количестве объектов в листе 10 равна  0.8087


### модель случайного дерева

In [11]:
best_model = None
best_result = 0
for est in range(1, 101,10):
    for depth in range(1,11): 
        for leaf in range(1,11):
            model_random_forrest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, min_samples_leaf=leaf) 
            model_random_forrest.fit(train_features, train_target ) 
            result = model_random_forrest.score(validate_features, validate_target)
            if result > best_result:
                best_model = model_random_forrest 
                best_result = result
print(best_model)
print("Accuracy наилучшей модели :", best_result)

RandomForestClassifier(max_depth=9, min_samples_leaf=3, n_estimators=61,
                       random_state=12345)
Accuracy наилучшей модели : 0.8164852255054432


доля правильных ответов самая высокая при глубине дерева 9, количестве деревьев 61,минимальном количестве объектов в листе 3 равна 0.792

пусть аргумент псевдослучайность будет случайным (random_state=None )

In [12]:
best_model = None
best_result = 0
for est in range(1, 101,10):
    for depth in range(1,11): 
        for leaf in range(1,11):
            model = RandomForestClassifier(n_estimators=est, max_depth=depth, min_samples_leaf=leaf) 
            model.fit(train_features, train_target ) 
            result = model.score(validate_features, validate_target)
            if result > best_result:
                best_model = model 
                best_result = result
print(best_model)
print("Accuracy наилучшей модели :", best_result)

RandomForestClassifier(max_depth=10, min_samples_leaf=2, n_estimators=21)
Accuracy наилучшей модели : 0.8195956454121306


доля правильных ответов увеличилась 

### модель логистической регресии 

In [13]:
best_model = None
best_result = 0
for iter in range(100,1500,200):
    model_logistic = LogisticRegression(random_state=12345, solver='lbfgs', max_iter= iter) 
    model_logistic.fit(train_features, train_target )
    result = model_logistic.score(validate_features, validate_target)
    if result > best_result:
        best_model = model_logistic 
        best_result = result
        print(best_model)
        print("Accuracy наилучшей модели на валидационной выборке:", best_result)

LogisticRegression(random_state=12345)
Accuracy наилучшей модели на валидационной выборке: 0.7013996889580093


доля правильных ответов довольно низкая, составляет 0,7

вывод: были рассмотрены три модели: решающего и случайного деревьев, логистической регрессии. наибольшая доля правильных ответов(accuracy) была достигнута в модели случайного дерева, равная 0.82. 

## Проверьте модель на тестовой выборке

проверим наши модели на тестовой выборке

In [14]:
#модель решающего дерева¶
predictions_valid = model_decision_tree.predict(test_features)
result = accuracy_score(test_target, predictions_valid)
print(result)

0.8009331259720062


In [15]:
 #модель случайного дерева¶

result = model_random_forrest.score(test_features, test_target)
print(result)

0.8087091757387247


In [16]:
 # модель логистической регресии 
result = model_logistic.score(test_features, test_target)
print(result)

0.6967340590979783


наибольшее значение доли правильных ответов на тестовой выборке получилось у модели случайного дерева

для анализа подходящего тарифа ультра стоит воспльзоваться моделью решающего дерева, которая наиболее точно предсказывает необходимый параметр is_ultra